# Import statements
Following packages are necessary:

In [1]:
import random
import utils
import closs
import os
import imageio
import json
from simplegen import SIMPLESequence

import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as kb
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint

from unet import UNet
from unet import preprocess_input as pre_une

from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from pdb import set_trace

# Checks
The version of tensorflow as well as the GPU support are checked.

In [2]:
print(tf.__version__)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.get_visible_devices('GPU')

2.2.0-rc3


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  # Disable all GPUS
  tf.config.set_visible_devices([], 'GPU')
  visible_devices = tf.config.get_visible_devices()
  for device in visible_devices:
    assert device.device_type != 'GPU'
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

print(tf.config.get_visible_devices('GPU'))
print(tf.config.get_visible_devices('CPU'))

[]
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


# Definition
Size definition is done here

In [4]:
width = 224
height = 224

grid_width = 5
grid_height = 5

grid_points = grid_width * grid_height

# Functionality
Necessary funcionality is added here

In [5]:
def eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path, multi_channel):
    validation_gen = SIMPLESequence(path_validation, path_fixed,
                                    batch_size=1,
                                    preprocess_input=pre_une,
                                    multi_channel=multi_channel)
    
    test_gen = SIMPLESequence(path_test, path_fixed,
                                batch_size=1,
                                preprocess_input=pre_une,
                                multi_channel=multi_channel)

    eu_loss = closs.EuclideanLoss(batch_size=1, loss_type='min')

    model.compile(optimizer='adam', loss = eu_loss)
    model.load_weights(store_path+'/weights.100.hdf5')

    warp_val = dict()
    maed_val_array = np.zeros(len(validation_gen))
    for index, val in enumerate(validation_gen):
        image_id = validation_gen._image_ids[index]
        
        if index % 100 == 0:
            print(index)
        X, y = val
        maed = model.evaluate(X,y,verbose=0)
        maed_val_array[index] = maed

        #y_pred = model.predict(X)
        #u_x = y_pred[0,:,:,0]
        #u_y = y_pred[0,:,:,1]

        #for inner_index in range(0,grid_points):
        #    x_pos = int(y[0, inner_index, 0, 0])
        #    y_pos = int(y[0, inner_index, 1, 0])

        #    ux_field = y_pred[0,:,:,0]
        #    uy_field = y_pred[0,:,:,1]

        #    ux = ux_field[y_pos][x_pos]
        #    uy = uy_field[y_pos][x_pos]

        #   x_pos = int(round(x_pos + ux))
        #    y_pos = int(round(y_pos + uy))

        #    warp_val[str(inner_index)] = [x_pos, y_pos]
        #    with open(store_path +'/val/{}_w.json'.format(image_id), 'w') as fp:
        #        json.dump(warp_val, fp)


    warp_test = dict()
    maed_test_array = np.zeros(len(test_gen))
    for index, val in enumerate(test_gen):
        image_id = test_gen._image_ids[index]
        
        if index % 100 == 0:
            print(index)
        X, y = val
        maed = model.evaluate(X,y,verbose=0)
        maed_test_array[index] = maed
        
        #y_pred = model.predict(X)
        #u_x = y_pred[0,:,:,0]
        #u_y = y_pred[0,:,:,1]

        #for inner_index in range(0,grid_points):
        #    x_pos = int(y[0, inner_index, 0, 0])
        #    y_pos = int(y[0, inner_index, 1, 0])

        #    ux_field = y_pred[0,:,:,0]
        #    uy_field = y_pred[0,:,:,1]

        #    ux = ux_field[y_pos][x_pos]
        #    uy = uy_field[y_pos][x_pos]

        #    x_pos = int(round(x_pos + ux))
        #    y_pos = int(round(y_pos + uy))

        #   warp_test[str(inner_index)] = [x_pos, y_pos]
        #    with open(store_path +'/test/{}_w.json'.format(image_id), 'w') as fp:
        #        json.dump(warp_test, fp)

    maed_array = np.concatenate((maed_val_array, maed_test_array))
    val_set = ['val' for i in range(len(validation_gen))]
    test_set = ['test' for i in range(len(test_gen))]
    set_type = val_set + test_set
    image_id = validation_gen._image_ids + test_gen._image_ids

    dataset = pd.DataFrame({'MIN': maed_array, 'Set': set_type, 'Image': image_id})
    dataset.to_csv(store_path + '/evaluation_min.csv')

# 6.4) Supervised Euclidean for SIMPLEA

In [6]:
store_path = 'experiments/6_4_euclidean'

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLEA/train/0'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLEA/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLEA/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLEA/test'

model = UNet(filters=32, layers=4, activation='linear', classes=2, input_shape=(224, 224, 1))

eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path, 'moving')

0
100
200
300
400
500
600
700
0
100
200
300
400
500
600
700


# 6.5) Supervised Euclidean for SIMPLEN

In [7]:
store_path = 'experiments/6_5_euclidean'

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLEN/train/0'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLEN/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLEN/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLEN/test'

model = UNet(filters=32, layers=4, activation='linear', classes=2, input_shape=(224, 224, 1))

eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path, 'moving')

0
100
200
300
400
500
600
700
0
100
200
300
400
500
600
700


# 6.6) Supervised Euclidean for SIMPLED

In [8]:
store_path = 'experiments/6_6_euclidean'

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train/0'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/test'

model = UNet(filters=32, layers=4, activation='linear', classes=2, input_shape=(224, 224, 1))

eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path, 'moving')

0
100
200
300
400
500
600
700
0
100
200
300
400
500
600
700


# 6.7) Supervised Euclidean for SIMPLED with mean absolute error

In [9]:
store_path = 'experiments/6_7_euclidean'

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train/0'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/test'

model = UNet(filters=32, layers=4, activation='linear', classes=2, input_shape=(224, 224, 1))

eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path, 'moving')

0
100
200
300
400
500
600
700
0
100
200
300
400
500
600
700


# 6.8) Supervised Euclidean for SIMPLED with fixed image

In [10]:
store_path = 'experiments/6_8_euclidean'

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train/0'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/test'

model = UNet(filters=32, layers=4, activation='linear', classes=2, input_shape=(224, 224, 2))

eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path, 'fixed')

0
100
200
300
400
500
600
700
0
100
200
300
400
500
600
700


# 6.9) Supervised Euclidean for SIMPLED with difference image

In [11]:
store_path = 'experiments/6_9_euclidean'

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train/0'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/test'

model = UNet(filters=32, layers=4, activation='linear', classes=2, input_shape=(224, 224, 2))

eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path, 'diff')

0
100
200
300
400
500
600
700
0
100
200
300
400
500
600
700


# 6.10) Supervised Euclidean for SIMPLED with difference image and gradient

In [12]:
store_path = 'experiments/6_10_euclidean'

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train/0'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/test'

model = UNet(filters=32, layers=4, activation='linear', classes=2, input_shape=(224, 224, 3))

eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path, 'grad')

/home/julian/anaconda3/envs/endolas/lib/python3.8/site-packages/skimage/filters/rank/generic.py:118: UserWarning: Possible precision loss converting image of type float32 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  image, selem, out, mask, n_bins = _handle_input(image, selem, out, mask,


0
100
200
300
400
500
600
700
0
100
200
300
400
500
600
700


# 6.11) Supervised Euclidean for SIMPLED with L1 regularization

In [13]:
store_path = 'experiments/6_11_euclidean'

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train/0'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/test'

kernel_regularizer=tf.keras.regularizers.l1(0.01)

model = UNet(filters=32,
             layers=4,
             activation='linear',
             classes=2,
             input_shape=(224, 224, 1),
             kernel_regularizer=kernel_regularizer)

eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path, 'moving')

0
100
200
300
400
500
600
700
0
100
200
300
400
500
600
700


# 6.12) Supervised Euclidean for SIMPLED with L2 regularization

In [14]:
store_path = 'experiments/6_12_euclidean'

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train/0'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/test'

kernel_regularizer=tf.keras.regularizers.l2(0.01)

model = UNet(filters=32,
             layers=4,
             activation='linear',
             classes=2,
             input_shape=(224, 224, 1),
             kernel_regularizer=kernel_regularizer)

eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path, 'moving')

0
100
200
300
400
500
600
700
0
100
200
300
400
500
600
700
